In [1]:
import numpy as np, pandas as pd

In [2]:
# load codebook
fname = 'https://github.com/aflaxman/SmartVA-Analyze-Mapping-Example/raw/master/Guide%20for%20data%20entry.xlsx'
cb = pd.read_excel(fname, index_col=2)
cb.head()

,module,question number,caption,data type
field name,,,,
sid,NaN,NaN,What is the study ID number?,text
gen_2_4,General,1.1,Address of/directions to household,text
gen_3_1,General,2.1,Did respondent give consent?,"yes = 1, no = 0"
gen_5_0,General,3.1,What was the name of the deceased?,text
gen_5_1a,General,3.2,What year was the deceased born?,integer


# Minimal example

Generate a .csv file that is accepted as input to SmartVA-Analyze 1.1

In [3]:
# SmartVA-Analyze 1.1 accepts a csv file as input
# and expects a column for every field name in the "Guide for data entry.xlsx" spreadsheet

df = pd.DataFrame(index=[0], columns=cb.index.unique())

In [4]:
# SmartVA-Analyze 1.1 also requires a handful of columns that are not in the Guide
df['child_3_10'] = np.nan
df['agedays']    = np.nan
df['child_5_7e'] = np.nan
df['child_5_6e'] = np.nan
df['adult_2_9a'] = np.nan

In [5]:
df.loc[0,'sid'] = 'example'

In [6]:
# if we save this dataframe as a csv, we can run it through SmartVA-Analyze 1.1

fname = 'example_1.csv'
df.to_csv(fname, index=False)

In [7]:
# here are the results of running this example through SmartVA-Analyze 1.1
pd.read_csv('neonate-predictions.csv')

,sid,cause,cause34,age,sex
0,example,NaN,Undetermined,0,NaN


# Example of simple, hypothetical mapping

If we have data on a set of verbal autopsies (VAs) that did not use the PHMRC Shortened Questionnaire, we must map them to the expected format.  This is a simple, hypothetical example for a set of VAs that asked only about injuries, hypertension, chest pain:

In [8]:
hypothetical_data = pd.DataFrame(index=range(5))

hypothetical_data['sex'] = ['M', 'M', 'F', 'M', 'F']
hypothetical_data['age'] = [35, 45, 75, 67, 91]

hypothetical_data['injury'] = ['rti', 'fall', '', '', '']
hypothetical_data['heart_disease'] = ['N', 'N', 'Y', 'Y', 'Y']
hypothetical_data['chest_pain'] = ['N', 'N', 'Y', 'N', '']

hypothetical_data

,sex,age,injury,heart_disease,chest_pain
0,M,35,rti,N,N
1,M,45,fall,N,N
2,F,75,,Y,Y
3,M,67,,Y,N
4,F,91,,Y,


In [9]:
# SmartVA-Analyze 1.1 accepts a csv file as input
# and expects a column for every field name in the "Guide for data entry.xlsx" spreadsheet

df = pd.DataFrame(index=hypothetical_data.index, columns=cb.index.unique())

# SmartVA-Analyze 1.1 also requires a handful of columns that are not in the Guide
df['child_3_10'] = np.nan
df['agedays']    = np.nan
df['child_5_7e'] = np.nan
df['child_5_6e'] = np.nan
df['adult_2_9a'] = np.nan

In [10]:
# to find the coding of specific variables, look in the Guide, and 
# as necessary refer to the numbers in paper form for the PHMRC Shortened Questionnaire
# http://www.healthdata.org/sites/default/files/files/Tools/SmartVA/2015/PHMRC%20Shortened%20VAI_all-modules_2015.zip

# set id
df['sid'] = hypothetical_data.index

# set sex
df['gen_5_2'] = hypothetical_data['sex'].map({'M': '1', 'F': '2'})

# set age
df['gen_5_4'] = 1  # units are years
df['gen_5_4a'] = hypothetical_data['age'].astype(int)


In [11]:
# good place to save work and confirm that it runs through SmartVA
fname = 'example_2.csv'
df.to_csv(fname, index=False)

In [12]:
# here are the results of running this example
pd.read_csv('adult-predictions.csv')

,sid,cause,cause34,age,sex
0,0,NaN,Undetermined,35,0
1,1,NaN,Undetermined,45,0
2,2,NaN,Undetermined,75,1
3,3,NaN,Undetermined,67,0
4,4,NaN,Undetermined,91,1


In [13]:
# map injuries to appropriate codes
# suffered injury?
df['adult_5_1'] = hypothetical_data['injury'].map({'rti':'1', 'fall':'1', '':'0'})
# injury type
df['adult_5_2'] = hypothetical_data['injury'].map({'rti':'1', 'fall':'2'})

In [14]:
# _another_ good place to save work and confirm that it runs through SmartVA
fname = 'example_3.csv'
df.to_csv(fname, index=False)

In [15]:
# here are the results of running this example
pd.read_csv('adult-predictions.csv')

,sid,cause,cause34,age,sex
0,0,30,Road Traffic,35,0
1,1,14,Falls,45,0
2,2,NaN,Undetermined,75,1
3,3,NaN,Undetermined,67,0
4,4,NaN,Undetermined,91,1


In [16]:
# map heart disease (to column adult_1_1i, see Guide)
df['adult_1_1i'] = hypothetical_data['heart_disease'].map({'Y':'1', 'N':'0'})

# map chest pain (to column adult_2_43, see Guide)
df['adult_2_43'] = hypothetical_data['chest_pain'].map({'Y':'1', 'N':'0', '':'9'})

In [18]:
# and that completes the work for a simple, hypothetical mapping
fname = 'example_4.csv'
df.to_csv(fname, index=False)

In [19]:
# have a look at the non-empty entries in the mapped database:
df.T.dropna()

,0,1,2,3,4
sid,0,1,2,3,4
gen_5_2,1,1,2,1,2
gen_5_4,1,1,1,1,1
gen_5_4a,35,45,75,67,91
adult_5_1,1,1,0,0,0
adult_1_1i,0,0,1,1,1
adult_2_43,0,0,1,0,9


In [20]:
# here are the results of running this example
pd.read_csv('adult-predictions.csv')

,sid,cause,cause34,age,sex
0,0,30,Road Traffic,35,0
1,1,14,Falls,45,0
2,2,17,IHD - Acute Myocardial Infarction,75,1
3,3,17,IHD - Acute Myocardial Infarction,67,0
4,4,17,IHD - Acute Myocardial Infarction,91,1
